In [ ]:
import json
from pathlib import Path
from typing import Mapping, Any, Optional

import numpy as np
import torch
from nn_core.common import PROJECT_ROOT
from torch import cosine_similarity
from tqdm import tqdm

from rae.modules.attention import RelativeAttention
from rae.modules.enumerations import (
    NormalizationMode,
    RelativeEmbeddingMethod,
    ValuesMethod,
    SimilaritiesQuantizationMode,
    AttentionOutput,
)
from rae.openfaiss import FaissIndex

In [ ]:
junk = {"English": "en", "Italian": "it", "Spanish": "es"}

In [ ]:
def _read_synset_info() -> Mapping[str, Mapping[str, Any]]:
    with (PROJECT_ROOT / "data" / "synset2lang2lemmas.tsv").open("r", encoding="utf-8") as fr:
        head = next(fr).strip().split("\t")
        langs = head[2:]
        langs = [junk.get(lang, lang) for lang in langs]
        synset2info = {}

        lang2all_lemmas = {}

        for i, line in enumerate(tqdm(fr, desc="Reading synset info")):
            # TODO: remove
            #             if i > 20_000_000:
            #                 break
            synset_id, synset_type, *lemmas = line.strip("\n").split("\t")
            if synset_type != "CONCEPT":
                continue

            assert len(langs) == len(lemmas)
            lang2lemma = dict(zip(langs, lemmas))

            lang2lemma = {lang: lang2lemma[lang] for lang in junk.values()}  # TODO: remove

            if any("_" in lemma for lang, lemma in lang2lemma.items()):
                continue

            if any(len(lemma) < 4 for lemma in lang2lemma.values()):
                continue

            if len(set(lang2lemma.values())) < len(lang2lemma.values()):
                continue

            for lang, lemma in lang2lemma.items():
                lang2all_lemmas.setdefault(lang, set())
                if lemma in lang2all_lemmas:
                    continue
                lang2all_lemmas[lang].add(lemma)

            synset2info[synset_id] = dict(synset_type=synset_type, lang2lemma=lang2lemma, synset_id=synset_id)

        return synset2info

In [ ]:
def build_anchors(
    lang2word2embedding: Mapping[str, Mapping[str, np.ndarray]], target_candidates: Optional[int] = 3_000
):
    synset_info = _read_synset_info()
    candidates = []

    for synset_id, info in tqdm(synset_info.items(), desc="Iterating synset info"):
        lang2lemma: Mapping[str, str] = info["lang2lemma"]
        if all(lang2word2embedding.get(lang, {}).get(lemma, None) is not None for lang, lemma in lang2lemma.items()):
            candidates.append(info)

        if target_candidates is not None and len(candidates) >= target_candidates:
            break

    lang2anchors = {}
    for candidate in candidates:
        for lang, lemma in candidate["lang2lemma"].items():
            lang2anchors.setdefault(lang, []).append(lemma)

    return lang2anchors

In [ ]:
def read_embeddings() -> Mapping[str, Mapping[str, np.ndarray]]:
    def read_file(file_path: Path, max_index: Optional[int] = 10_000) -> Mapping[str, np.ndarray]:
        with file_path.open("r", encoding="utf-8") as fr:
            next(fr)

            word2embedding = {}
            for i, line in enumerate(tqdm(fr, desc=f"Reading {file_path}")):
                if max_index is not None and i > max_index:
                    break
                word, *embedding = line.strip().split(" ")
                embedding = np.array([float(x) for x in embedding])
                word2embedding[word] = embedding

            return word2embedding

    lang2word2embedding = {}

    for file in (PROJECT_ROOT / "fasttext").iterdir():
        if ".gz" in file.suffixes:
            continue
        lang = file.suffixes[0].strip(".")
        word2embedding = read_file(file_path=file, max_index=20_000)
        lang2word2embedding[lang] = word2embedding

    return lang2word2embedding

In [ ]:
device: str = "cuda"

In [ ]:
lang2word2embedding = read_embeddings()

In [ ]:
abs_lang2faiss_index = {lang: FaissIndex(d=300) for lang, word2embedding in lang2word2embedding.items()}
for lang, faiss_index in abs_lang2faiss_index.items():
    faiss_index: FaissIndex
    faiss_index.add_vectors(embeddings=lang2word2embedding[lang].items(), normalize=True)

In [ ]:
lang2anchors = build_anchors(lang2word2embedding=lang2word2embedding, target_candidates=1000)
Path(PROJECT_ROOT / "lang2anchors.json").write_text(json.dumps(lang2anchors, indent=4))
n_anchors: int = len(list(lang2anchors.values())[0])
n_anchors

In [ ]:
attention_block: RelativeAttention = RelativeAttention(
    in_features=300,
    hidden_features=None,
    n_anchors=n_anchors,
    n_classes=None,
    normalization_mode=NormalizationMode.L2,
    similarity_mode=RelativeEmbeddingMethod.INNER,
    values_mode=ValuesMethod.SIMILARITIES,
    similarities_quantization_mode=SimilaritiesQuantizationMode.DIFFERENTIABLE_ROUND,
    similarities_bin_size=0.5,
    #     similarities_quantization_mode=None,
    #     similarities_bin_size=None,
    similarities_aggregation_mode=None,
    similarities_aggregation_n_groups=None,
    anchors_sampling_mode=None,
    n_anchors_sampling_per_class=None,
).to(device)

In [ ]:
rel_lang2faiss_index = {lang: FaissIndex(d=n_anchors) for lang, _ in lang2word2embedding.items()}
lang2anchor_embeddings = {
    lang: torch.stack([torch.tensor(lang2word2embedding[lang][anchor]).to(device) for anchor in anchors])
    for lang, anchors in lang2anchors.items()
}
for lang, faiss_index in rel_lang2faiss_index.items():
    word2embedding = lang2word2embedding[lang]
    words, embeddings = list(zip(*word2embedding.items()))
    embeddings = torch.tensor(embeddings).to(device)
    embeddings = attention_block(x=embeddings, anchors=lang2anchor_embeddings[lang])[AttentionOutput.OUTPUT]
    faiss_index.add_vectors(embeddings=list(zip(words, embeddings.cpu().numpy())), normalize=True)

In [ ]:
word_en: str = "shampoo"
word_it: str = "shampoo"
word_en_rel_vector = rel_lang2faiss_index["en"].reconstruct(word_en)
word_en_abs_vector = abs_lang2faiss_index["en"].reconstruct(word_en)
word_it_rel_vector = rel_lang2faiss_index["it"].reconstruct(word_it)
word_it_abs_vector = abs_lang2faiss_index["it"].reconstruct(word_it)

In [ ]:
abs_lang2faiss_index["en"].search_by_keys(query=[word_en], k_most_similar=10)

In [ ]:
rel_lang2faiss_index["en"].search_by_keys(query=[word_en], k_most_similar=10)

In [ ]:
rel_lang2faiss_index["it"].search_by_keys(query=[word_it], k_most_similar=10)

In [ ]:
rel_lang2faiss_index["it"].search_by_vectors(
    query_vectors=np.array([word_en_rel_vector], dtype="float32"),
    k_most_similar=10,
    normalize=False,
)

In [ ]:
rel_lang2faiss_index["en"].search_by_vectors(
    query_vectors=np.array([word_it_rel_vector], dtype="float32"),
    k_most_similar=10,
    normalize=False,
)

In [ ]:
cosine_similarity(
    x1=torch.tensor(word_en_rel_vector),
    x2=torch.tensor(word_it_rel_vector),
    dim=-1,
)

In [ ]:
cosine_similarity(
    x1=torch.tensor(word_en_abs_vector),
    x2=torch.tensor(word_it_abs_vector),
    dim=-1,
)

In [ ]:
rel_diff = (torch.tensor(word_en_rel_vector) - torch.tensor(word_it_rel_vector)).abs().sum()
rel_diff, rel_diff / n_anchors

In [ ]:
abs_diff = (torch.tensor(word_en_abs_vector) - torch.tensor(word_it_abs_vector)).abs().sum()
abs_diff, abs_diff / 300